# 1. Algorithm

In [1]:
def algo_solution(nums, target):
    # use hash table
    dict = {}
    for i in range(len(nums)):
        if target - nums[i] in dict:
            return [dict[target - nums[i]], i]
        dict[nums[i]] =  i

Run example 1

In [2]:
nums = [2,7,11,15]
target = 9
algo_solution(nums, target)

[0, 1]

Run example 3

In [3]:
nums = [3,3]
target = 6
algo_solution(nums, target)

[0, 1]

# 2. Data Science

In [4]:
# libraries
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
import random

## Load data and have a look

In [5]:
# check data
df=pd.read_csv("input.csv")
df.head()

,ab_test_version,activity_kind,ad_impressions_count,ad_mediation_platform,ad_revenue_network,ad_revenue_unit,adgroup_name,adid,app_version_short,att_status,...,idfa_android_id,idfa_gps_adid,installed_at,is_organic,network_name,os_name,reporting_revenue,session_count,sk_conversion_value,sub_details
0,38,session,0,NaN,NaN,NaN,969da0d8a7b4aff20da9d6482dd4cfbd,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,...,NaN,NaN,2021-11-08 00:19:21.000,0,applovin,ios,0.0,4,0,NaN
1,38,event,0,NaN,NaN,NaN,969da0d8a7b4aff20da9d6482dd4cfbd,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,...,NaN,NaN,2021-11-08 00:19:21.000,0,applovin,ios,0.0,4,0,NaN
2,38,event,0,NaN,NaN,NaN,969da0d8a7b4aff20da9d6482dd4cfbd,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,...,NaN,NaN,2021-11-08 00:19:21.000,0,applovin,ios,0.0,4,0,NaN
3,29,session,0,NaN,NaN,NaN,NaN,220db1e74489be3928b976acb062a75f,2.7.4,2,...,NaN,NaN,2021-08-01 15:26:32.000,1,Organic,ios,0.0,13,0,NaN
4,29,event,0,NaN,NaN,NaN,NaN,220db1e74489be3928b976acb062a75f,2.7.4,2,...,NaN,NaN,2021-08-01 15:26:32.000,1,Organic,ios,0.0,13,0,NaN


In [6]:
print(df.shape)
print(df.dtypes)

(2104313, 30)
ab_test_version            int64
activity_kind             object
ad_impressions_count       int64
ad_mediation_platform     object
ad_revenue_network        object
ad_revenue_unit           object
adgroup_name              object
adid                      object
app_version_short         object
att_status                 int64
campaign_name             object
conversion_duration        int64
country                   object
country_subdivision       object
created_at                object
creative_name             object
device_manufacturer       object
device_name               object
device_type               object
event_name                object
idfa_android_id           object
idfa_gps_adid             object
installed_at              object
is_organic                 int64
network_name              object
os_name                   object
reporting_revenue        float64
session_count              int64
sk_conversion_value        int64
sub_details               obj

In [7]:
df.columns

Index(['ab_test_version', 'activity_kind', 'ad_impressions_count',
       'ad_mediation_platform', 'ad_revenue_network', 'ad_revenue_unit',
       'adgroup_name', 'adid', 'app_version_short', 'att_status',
       'campaign_name', 'conversion_duration', 'country',
       'country_subdivision', 'created_at', 'creative_name',
       'device_manufacturer', 'device_name', 'device_type', 'event_name',
       'idfa_android_id', 'idfa_gps_adid', 'installed_at', 'is_organic',
       'network_name', 'os_name', 'reporting_revenue', 'session_count',
       'sk_conversion_value', 'sub_details'],
      dtype='object')

In [8]:
# select numeric columns
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)

# select non numeric columns
df_non_numeric = df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print(non_numeric_cols)

['ab_test_version' 'ad_impressions_count' 'att_status'
 'conversion_duration' 'is_organic' 'reporting_revenue' 'session_count'
 'sk_conversion_value']
['activity_kind' 'ad_mediation_platform' 'ad_revenue_network'
 'ad_revenue_unit' 'adgroup_name' 'adid' 'app_version_short'
 'campaign_name' 'country' 'country_subdivision' 'created_at'
 'creative_name' 'device_manufacturer' 'device_name' 'device_type'
 'event_name' 'idfa_android_id' 'idfa_gps_adid' 'installed_at'
 'network_name' 'os_name' 'sub_details']


In [9]:
df['reporting_revenue'].describe()
# negative revenue ? but small value -> might have a reason
# small revenue by action, at most 17 dollars

count    2.104313e+06
mean     1.031772e-02
std      2.067289e-01
min     -1.534000e-02
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.766900e+01
Name: reporting_revenue, dtype: float64

## Some thoughts
We want to predict the LVT by user, thus the final revenue value is to be grouped by user.
Two ways to do this:    
  
  * we predict directly the LVT by user, but need to process all features, and we might lose some information
  * we predict the event revenue, and finally sum them up (**we start with this approach which might make use of more data**)
    

### Missing Data Percentage List

In [10]:
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

ab_test_version - 0%
activity_kind - 0%
ad_impressions_count - 0%
ad_mediation_platform - 77%
ad_revenue_network - 77%
ad_revenue_unit - 77%
adgroup_name - 48%
adid - 16%
app_version_short - 20%
att_status - 0%
campaign_name - 47%
conversion_duration - 0%
country - 9%
country_subdivision - 31%
created_at - 0%
creative_name - 49%
device_manufacturer - 45%
device_name - 32%
device_type - 7%
event_name - 59%
idfa_android_id - 64%
idfa_gps_adid - 64%
installed_at - 0%
is_organic - 0%
network_name - 1%
os_name - 5%
reporting_revenue - 0%
session_count - 0%
sk_conversion_value - 0%
sub_details - 99%


Some user ids are missing too, we eliminate these observations.

In [11]:
# drop rows with a lot of missing values (>0%)
cols_to_drop = ['sub_details', 'idfa_gps_adid', 'idfa_android_id', 'event_name', 
                'device_manufacturer', 'creative_name', 'adgroup_name', 
                'ad_mediation_platform', 'ad_revenue_network', 'ad_revenue_unit']
df_p = df.drop(cols_to_drop, axis=1)
df_p = df_p.loc[df_p['adid'].notnull(), :]
df_p

,ab_test_version,activity_kind,ad_impressions_count,adid,app_version_short,att_status,campaign_name,conversion_duration,country,country_subdivision,created_at,device_name,device_type,installed_at,is_organic,network_name,os_name,reporting_revenue,session_count,sk_conversion_value
0,38,session,0,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,PLAW_iOS_CPI (023cde1d4b966315b8ad1227f0915f00),557,us,New Jersey,2021-11-27 05:02:25.000,iPad,tablet,2021-11-08 00:19:21.000,0,applovin,ios,0.000,4,0
1,38,event,0,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,PLAW_iOS_CPI (023cde1d4b966315b8ad1227f0915f00),557,us,New Jersey,2021-11-27 05:02:25.000,iPad,tablet,2021-11-08 00:19:21.000,0,applovin,ios,0.000,4,0
2,38,event,0,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,PLAW_iOS_CPI (023cde1d4b966315b8ad1227f0915f00),557,us,New Jersey,2021-11-27 05:02:50.000,iPad,tablet,2021-11-08 00:19:21.000,0,applovin,ios,0.000,4,0
3,29,session,0,220db1e74489be3928b976acb062a75f,2.7.4,2,NaN,0,us,Oklahoma,2021-11-27 05:04:04.000,iPhone12ProMax,phone,2021-08-01 15:26:32.000,1,Organic,ios,0.000,13,0
4,29,event,0,220db1e74489be3928b976acb062a75f,2.7.4,2,NaN,0,us,Oklahoma,2021-11-27 05:04:04.000,iPhone12ProMax,phone,2021-08-01 15:26:32.000,1,Organic,ios,0.000,13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104300,38,event,0,68618444c182deee0fdec3dadaa43d56,2.9.2,2,LWP IOS US_8383098,400,us,Georgia,2021-10-30 18:58:33.000,iPhoneSE(2ndgeneration),phone,2021-10-30 18:26:13.000,0,Ironsource,ios,0.000,1,0
2104303,38,session,0,b6e523c361922ccda1f6971ccb58cb67,2.9.2,1,LWP IOS UK_8447539,232,gb,England,2021-10-30 18:59:17.000,iPhoneSE,phone,2021-10-30 16:30:50.000,0,Ironsource,ios,0.000,2,0
2104304,36,session,0,4bf97a4c2edc4a6723f4afdb6527e88d,2.9.2,3,LWP IOS US / CA / NZ / AU (13173376314),98,au,New South Wales,2021-10-30 18:59:21.000,iPhone8Plus,phone,2021-08-10 11:02:31.000,0,Google Ads ACI,ios,0.000,11,0
2104305,36,event,0,4bf97a4c2edc4a6723f4afdb6527e88d,2.9.2,3,LWP IOS US / CA / NZ / AU (13173376314),98,au,New South Wales,2021-10-30 18:59:21.000,iPhone8Plus,phone,2021-08-10 11:02:31.000,0,Google Ads ACI,ios,0.000,11,0


In [12]:
# impute the missing values and create the missing value indicator variables for each numeric column.
df_numeric = df_p.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

for col in numeric_cols:
    missing = df_p[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print('imputing missing values for: {}'.format(col))
        df_p['{}_ismissing'.format(col)] = missing
        med = df_p[col].median()
        df_p[col] = df_p[col].fillna(med)
        
# impute the missing values and create the missing value indicator variables for each non-numeric column.
df_non_numeric = df_p.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values

for col in non_numeric_cols:
    missing = df_p[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print('imputing missing values for: {}'.format(col))
        df_p['{}_ismissing'.format(col)] = missing
        
        top = df[col].describe()['top'] # impute with the most frequent value.
        df_p[col] = df_p[col].fillna(top)

imputing missing values for: app_version_short
imputing missing values for: campaign_name
imputing missing values for: country
imputing missing values for: country_subdivision
imputing missing values for: device_name
imputing missing values for: device_type
imputing missing values for: network_name
imputing missing values for: os_name


In [13]:
num_rows = len(df_p.index)
low_information_cols = [] #

for col in df_p.columns:
    cnts = df_p[col].value_counts(dropna=False)
    top_pct = (cnts/num_rows).iloc[0]
    if top_pct > 0.95:
        low_information_cols.append(col)
        print('{0}: {1:.5f}%'.format(col, top_pct*100))
        print(cnts)
        print()

os_name: 100.00000%
ios    1774754
Name: os_name, dtype: int64

sk_conversion_value: 100.00000%
0    1774754
Name: sk_conversion_value, dtype: int64

app_version_short_ismissing: 95.41091%
False    1693309
True       81445
Name: app_version_short_ismissing, dtype: int64

device_type_ismissing: 95.37530%
False    1692677
True       82077
Name: device_type_ismissing, dtype: int64

network_name_ismissing: 99.99301%
False    1774630
True         124
Name: network_name_ismissing, dtype: int64

os_name_ismissing: 95.41091%
False    1693309
True       81445
Name: os_name_ismissing, dtype: int64



In [14]:
df_p = df_p.drop(['os_name', 'sk_conversion_value'], axis=1)

In [15]:
print(df_p.columns)
df_p.head()

Index(['ab_test_version', 'activity_kind', 'ad_impressions_count', 'adid',
       'app_version_short', 'att_status', 'campaign_name',
       'conversion_duration', 'country', 'country_subdivision', 'created_at',
       'device_name', 'device_type', 'installed_at', 'is_organic',
       'network_name', 'reporting_revenue', 'session_count',
       'app_version_short_ismissing', 'campaign_name_ismissing',
       'country_ismissing', 'country_subdivision_ismissing',
       'device_name_ismissing', 'device_type_ismissing',
       'network_name_ismissing', 'os_name_ismissing'],
      dtype='object')


,ab_test_version,activity_kind,ad_impressions_count,adid,app_version_short,att_status,campaign_name,conversion_duration,country,country_subdivision,...,reporting_revenue,session_count,app_version_short_ismissing,campaign_name_ismissing,country_ismissing,country_subdivision_ismissing,device_name_ismissing,device_type_ismissing,network_name_ismissing,os_name_ismissing
0,38,session,0,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,PLAW_iOS_CPI (023cde1d4b966315b8ad1227f0915f00),557,us,New Jersey,...,0.0,4,False,False,False,False,False,False,False,False
1,38,event,0,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,PLAW_iOS_CPI (023cde1d4b966315b8ad1227f0915f00),557,us,New Jersey,...,0.0,4,False,False,False,False,False,False,False,False
2,38,event,0,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,PLAW_iOS_CPI (023cde1d4b966315b8ad1227f0915f00),557,us,New Jersey,...,0.0,4,False,False,False,False,False,False,False,False
3,29,session,0,220db1e74489be3928b976acb062a75f,2.7.4,2,PLAW_iOS_CPI (023cde1d4b966315b8ad1227f0915f00),0,us,Oklahoma,...,0.0,13,False,True,False,False,False,False,False,False
4,29,event,0,220db1e74489be3928b976acb062a75f,2.7.4,2,PLAW_iOS_CPI (023cde1d4b966315b8ad1227f0915f00),0,us,Oklahoma,...,0.0,13,False,True,False,False,False,False,False,False


In [16]:
df_p["activity_kind"] = df_p["activity_kind"].astype('category')
df_p["campaign_name"] = df_p["campaign_name"].astype('category')
df_p["country_subdivision"] = df_p["country_subdivision"].astype('category')
df_p["country"] = df_p["country"].astype('category')
df_p["device_name"] = df_p["device_name"].astype('category')
df_p["device_type"] = df_p["device_type"].astype('category')
df_p['network_name'] = df_p["network_name"].astype('category')
df_p["installed_at"] = pd.to_datetime(df_p["installed_at"])
df_p["created_at"] = pd.to_datetime(df_p["created_at"])
df_p["action_time_after_install"] = df_p['created_at'] - df_p['installed_at']
df_p["action_time_after_install"] = df_p['action_time_after_install'].dt.days
df_p['entry_time'] = df_p['installed_at'] - df_p['installed_at'].min()
df_p["entry_time"] = df_p['entry_time'].dt.days
df_p = df_p.drop(['installed_at', 'created_at'], axis=1)
df_p['app_version_short'] = df_p['app_version_short'].astype(str).str[0].astype('category')


In [17]:
df_p.dtypes

ab_test_version                     int64
activity_kind                    category
ad_impressions_count                int64
adid                               object
app_version_short                category
att_status                          int64
campaign_name                    category
conversion_duration                 int64
country                          category
country_subdivision              category
device_name                      category
device_type                      category
is_organic                          int64
network_name                     category
reporting_revenue                 float64
session_count                       int64
app_version_short_ismissing          bool
campaign_name_ismissing              bool
country_ismissing                    bool
country_subdivision_ismissing        bool
device_name_ismissing                bool
device_type_ismissing                bool
network_name_ismissing               bool
os_name_ismissing                 

In [18]:
enc = OrdinalEncoder()
df_pt = df_p.drop(['adid'], axis=1, inplace=False)
all_cols = df_pt.columns
df_pt[all_cols] = enc.fit_transform(df_pt[all_cols])

In [19]:
# Try random forest which is tolerent with categorical variables without problem of encoding

# train - test split
# split by user
unique_users = df_p['adid'].unique()
random.shuffle(unique_users)
len(unique_users) * 0.8
# 217096 unique users

173676.80000000002

In [20]:
train_users = unique_users[:173676]
print(len(train_users))
test_users = unique_users[173676:]
print(len(test_users))

173676
43420


In [21]:
train_X = df_pt.loc[df_p['adid'].isin(train_users), :]
train_Y = df_p.loc[df_p['adid'].isin(train_users), 'reporting_revenue']

test_X = df_pt.loc[df_p['adid'].isin(test_users), :]
test_Y = df_p.loc[df_p['adid'].isin(test_users), 'reporting_revenue']


print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

(1418932, 25)
(1418932,)
(355822, 25)
(355822,)


## Random forest approach

In [22]:
regr = RandomForestRegressor(random_state=0) # without tuning, take default mtry, good try already
regr.fit(train_X, train_Y)
with open('my_random_forest.pkl', 'wb') as f:
    pickle.dump(regr, f) 

In [23]:
result_rf = test_X
result_rf['prediction'] = regr.predict(test_X)
result_rf['truth'] = test_Y
result_rf['adid'] = df_p.loc[df_p['adid'].isin(test_users), 'adid']
prev = result_rf.groupby('adid').sum()['prediction']
truth = result_rf.groupby('adid').sum()['truth']

print(mean_squared_error(truth, prev))

/var/folders/nv/njmv5dbn2sb54w1y8g4pphbr0000gn/T/ipykernel_64216/1421986951.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_rf['prediction'] = regr.predict(test_X)
/var/folders/nv/njmv5dbn2sb54w1y8g4pphbr0000gn/T/ipykernel_64216/1421986951.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_rf['truth'] = test_Y
/var/folders/nv/njmv5dbn2sb54w1y8g4pphbr0000gn/T/ipykernel_64216/1421986951.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

0.00030655288472759525


## Gradient Boosting

In [ ]:
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.1,
    "loss": "squared_error",
}
gb = GradientBoostingRegressor(**params)
gb.fit(train_X, train_Y)

In [ ]:
# NOT RUN, not enough time to finish
result_gb = test_X
result_gb['prediction'] = regr.predict(test_X)
result_gb['truth'] = test_Y
result_gb['adid'] = df_p.loc[df_p['adid'].isin(test_users), 'adid']
prev = result_gb.groupby('adid').sum()['prediction']
truth = result_gb.groupby('adid').sum()['truth']

print(mean_squared_error(truth, prev))

## Final words
We could also try other encoding methods, or more encapsulated solutions like h2o which treats categorical variables automatically. SVM, neural networks or elastic net models could also be explored.